In [ ]:
# Test performance on production W&B UI style queries.

import cProfile
import weave
weave.use_fixed_server_port()
import os

In [ ]:
# An artifact page table query. There are various sized tables in this project at different versions.

x = (
    weave.ops.project("shawn", "dsviz-simple-tables")
    .artifact("simple_tables")
    .membershipForAlias("v5")
    .artifactVersion()
    .file("my-table.table.json")
    .table()
    .rows()
    .createIndexCheckpointTag()
    .groupby(lambda row: weave.ops.dict_(x=row["x"]))[0]['c']
#     .sort(lambda group: weave.ops.list(a=group.groupkey()["x"]), ["asc"])[0]["c"]
)

In [ ]:
# A run page concat query. There is also log-tables-200k-100.

x = (weave.ops.project("shawn", "log-tables-10k-20")
  .filteredRuns("{\"name\":{\"$ne\":null}}", "-createdAt")
  .limit(50)
  .summary()['tab1']
  .table()
  .rows()
  .dropna()
  .concat()
  .createIndexCheckpointTag()
  [0]['config']['key0'])

In [ ]:
weave.use(x)

In [ ]:
# same as above but run in profiler

cProfile.run(
"""
x = (
    weave.ops.project("shawn", "dsviz-simple-tables")
    .artifact("simple_tables")
    .membershipForAlias("v5")
    .artifactVersion()
    .file("my-table.table.json")
    .table()
    .rows()
    .createIndexCheckpointTag()
    .groupby(lambda row: weave.ops.dict_(x=row["x"]))[0]['c']
#     .sort(lambda group: weave.ops.list(a=group.groupkey()["x"]), ["asc"])[0]["c"]
)
""", 'profile_stats'
)

In [ ]:
# inspect saved profile data

import pstats
from pstats import SortKey
p = pstats.Stats('profile_stats')
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats()
#p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_callers('type_of')
#p.strip_dirs().sort_stats(SortKey.TIME).print_stats()